## Biot's equation

This notebooks aims at testing the effect of different pressure boundary conditions for Biot's equation on a 2D doughnut mesh with total stress boundary conditions and a pressure gradient between the outer and inner boundary.

The quasi-static equations for Biot's model are:

$$
- \nabla \cdot 2 \mu \epsilon (u) - \nabla \lambda \nabla u I + \alpha \nabla p = f \quad \text{in $\Omega \times (0,T]$ (momentum equation)}
$$
$$
c \frac{\partial p}{\partial t} + \alpha \frac{\nabla \cdot u}{\partial t} - \nabla \cdot (K \nabla p) = g \quad \text{ in $\Omega \times (0,T]$ (continuity equation)}
$$

Suitable boundary conditions might be either No-flux (Neumann) or Dirichlet BCs for the pressure and a total stress BC for the displacement:

$$
(2 \mu \epsilon (u) + \lambda \nabla \cdot u I - \alpha p I) \cdot n = - p_{obs} n \quad \text{on $\Gamma \times (0,T]$}
$$

$$
\nabla p \cdot n = 0 \quad \text{on $\Gamma_N \times (0,T]$} \\
p = p_{obs} \quad \text{on $\Gamma_D \times (0,T]$}
$$

As inital conditions, we assume the system to be at rest:

$$
u(x, 0) = 0 \text{ in $\Omega$}\\
p(x, 0) = 0 \text{ in $\Omega$}
$$



In [ ]:
%load_ext autoreload
%autoreload 2
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from ufl import Identity, nabla_div
import ufl
#from fenics_adjoint import *
from braininversion.BiotSolver import solve_biot
from braininversion.meshes import generate_doughnut_mesh
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                            plot_pressures_and_forces_cross_section,
                            extract_cross_section, style_dict)


T = 0.1           # final time
num_steps = 1    # number of time steps
dt = T / num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 1e-17       # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-8         # storage coefficent
alpha = 1.0         # Biot-Willis coefficient

# Biot material parameters
E = 1500.0          # Young modulus
nu = 0.479         # Poisson ratio

material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K
material_parameter["lmbda"] = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
material_parameter["mu"] = E/(2.0*(1.0+nu))
material_parameter["alpha"] = alpha

brain_radius = 0.1
ventricle_radius = 0.05
N = 100
mmHg2Pa = 132.32
freq = 1.0 
A = 0.2*mmHg2Pa
p_obs_outer = Expression("A", A=A,f=freq,t=0,degree=2)
p_obs_inner = Constant(0.0)

u_nullspace = True

g = Constant(0.0)
f = Constant([0.0, 0.0])

x_coords = np.linspace(ventricle_radius, brain_radius, 100)
slice_points = [Point(x, 0.0) for x in x_coords]

In [ ]:
# total stress at both boundaries, no flux BC for pressure

res = [4,8,16, 32, 64]
mu = material_parameter["mu"]
lmbda = material_parameter["lmbda"]
I = Identity(2)

def eps(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

def sigma(u):
    return lmbda*nabla_div(u)*I + 2*mu*eps(u)

def compute_normal_stress_convergence(boundary_conditions_p):

    boundary_id = 1
    i = 0
    normal_stresses = []
    tang_stresses = []
    pressure_results = []
    h_max = []
    for N in  res:
        mesh, bm = generate_doughnut_mesh(brain_radius, ventricle_radius, N)
        n = FacetNormal(mesh)

        boundary_conditions_u = {2:{"Neumann":n*p_obs_outer},
                                 1:{"Neumann":n*p_obs_inner},
                                 #2:{"Neumann":n*p_obs_lin},
                            } 

        ds = Measure("ds", domain=mesh, subdomain_data=bm)
        tangent = as_vector([n[1], -n[0]])
        solution = solve_biot(mesh, f, g, T, num_steps, material_parameter,
                              bm, boundary_conditions_p,
                              bm, boundary_conditions_u,
                              u_nullspace=u_nullspace, theta=1.0,
                              u_degree=2, p_degree=1)
        solution = [s.copy() for s in solution]
        V = FunctionSpace(mesh, "CG", 1)
        displ = [s.split()[0] for s in solution]
        total_pressure = [s.split()[1] for s in solution]
        pressure = [s.split()[2] for s in solution]
        lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]

        p = pressure[i]
        u = displ[i]
        pT = total_pressure[i]
        total_stress = sigma(u) - alpha*p*I
        normal_tot_stress = assemble(sqrt(inner(dot(total_stress, n), dot(total_stress, n))) * ds(boundary_id))
        tang_tot_stress = assemble(sqrt(inner(dot(total_stress, tangent),
                                         dot(total_stress, tangent))) * ds(boundary_id))
        normal_stresses.append(normal_tot_stress)
        tang_stresses.append(tang_tot_stress)
        h_max.append(mesh.hmax())
        pressure_results.append(p.copy())
    return pressure_results, normal_stresses, tang_stresses, solution, mesh
        


In [ ]:
# Compute normal stress convergence for no-flux pressure BCs
boundary_conditions_p = {2:{"Neumann":Constant(0.0)},
                         1:{"Neumann":Constant(0.0)},
                         }

pressure_results, normal_stresses, tang_stresses, solution, mesh = compute_normal_stress_convergence(boundary_conditions_p)

plt.figure(figsize=(9,7))
plt.plot(res, normal_stresses, "*-",label="normal [N]" )
plt.plot(res, tang_stresses, "*-",label="tangential [N]" )

plt.xlabel("refinement level N")
plt.ylabel("stress on inner boundary (L2)")
plt.legend()
plt.grid()

In [ ]:
plt.figure(figsize=(15,8))
for i, p in enumerate(pressure_results):
    plt.subplot(2, len(pressure_results)/2 + 1, i + 1)
    c = plot(p)
    cb = plt.colorbar(c)
    cb.set_label("$p_F$ [Pa]")
    plt.title(f"N = {res[i]} \n stress on inner boundary: {normal_stresses[i]:.2f} [N]")
plt.subplots_adjust(hspace=0.3, wspace=0.3)

In [ ]:
V = FunctionSpace(mesh, "CG", 1)
displ = [s.split()[0] for s in solution]
total_pressure = [s.split()[1] for s in solution]
pressure = [s.split()[2] for s in solution]
lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]

x_coords = np.linspace(ventricle_radius, brain_radius, 100)
slice_points = [Point(x, 0.0) for x in x_coords]

pressures = {"negative_total_pressure" : extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
             "fluid_pressure" : extract_cross_section(pressure, slice_points)/mmHg2Pa,
             "lambda_div_u"   : extract_cross_section(lmbda_div_u, slice_points)/mmHg2Pa,
             "prescribed outer pressure"   : extract_cross_section(p_obs_outer, slice_points, times)/mmHg2Pa,}

displacement = {"displacement [m]": extract_cross_section(displ, slice_points),}
style_dict["negative_total_pressure"] = {"ls":":", "lw":3, "color":"firebrick"}
style_dict["fluid_pressure"] = {"ls":":", "lw":3, "color":"orange"}
style_dict["displacement [m]"] = {"ls":"-.", "lw":3, "color":"green"}
style_dict["p_opt"] = {"ls":"-.", "lw":3, "color":"blue"}
style_dict["prescribed outer pressure"] = {"ls":"-.", "lw":3, "color":"navy"}

for i in [0]: #range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, displacement, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f} s")

In [ ]:
# Compute normal stress convergence for Dirichlet BCs
boundary_conditions_p = {2:{"Dirichlet":p_obs_outer},
                         1:{"Dirichlet":p_obs_inner},
                         }

pressure_results, normal_stresses, tang_stresses, solution, mesh = compute_normal_stress_convergence(boundary_conditions_p)

plt.figure(figsize=(9,7))
plt.plot(res, normal_stresses, "*-",label="normal [N]" )
plt.plot(res, tang_stresses, "*-",label="tangential [N]" )

plt.xlabel("refinement level N")
plt.ylabel("stress on inner boundary (L2)")
plt.legend()
plt.grid()

In [ ]:
plt.figure(figsize=(15,8))
for i, p in enumerate(pressure_results):
    plt.subplot(2, len(pressure_results)/2 + 1, i + 1)
    c = plot(p)
    cb = plt.colorbar(c)
    cb.set_label("$p_F$ [Pa]")
    plt.title(f"N = {res[i]} \n stress on inner boundary: {normal_stresses[i]:.2f} [N]")
plt.subplots_adjust(hspace=0.3, wspace=0.3)

In [ ]:
V = FunctionSpace(mesh, "CG", 1)
displ = [s.split()[0] for s in solution]
total_pressure = [s.split()[1] for s in solution]
pressure = [s.split()[2] for s in solution]
lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]

x_coords = np.linspace(ventricle_radius, brain_radius, 100)
slice_points = [Point(x, 0.0) for x in x_coords]

pressures = {"negative_total_pressure" : extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
             "fluid_pressure" : extract_cross_section(pressure, slice_points)/mmHg2Pa,
             "lambda_div_u"   : extract_cross_section(lmbda_div_u, slice_points)/mmHg2Pa,
             "prescribed outer pressure"   : extract_cross_section(p_obs_outer, slice_points, times)/mmHg2Pa,}

displacement = {"displacement [m]": extract_cross_section(displ, slice_points),}

for i in [0]: #range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, displacement, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f} s")